In [ ]:
from ctapipe.calib import CameraCalibrator
from ctapipe.image import tailcuts_clean
from ctapipe.io import EventSource

from lstchain.calib.camera.calibration_calculator import CalibrationCalculator

from astropy.coordinates import EarthLocation
import astropy.units as u

from traitlets.config import Config

import numpy as np
from copy import deepcopy

telescope_location = EarthLocation(lon=342.108612 * u.deg,
                                   lat=28.761389 * u.deg,
                                   height=2147 * u.m)
config_data =Config(
    {"LSTEventSource": {
        "min_flatfield_adc": 3000,
        "min_flatfield_pixel_fraction": 0.8,
        "calibrate_flatfields_and_pedestals": False,
        "LSTR0Corrections" : {
            "drs4_pedestal_path" : "/fefs/aswg/data/real/DL1/20220409/v0.9/tailcut84/log/drs4_pedestal.Run07759.0000.h5",
            "calibration_path" : "/fefs/aswg/data/real/DL1/20220409/v0.9/tailcut84/log/calibration_filters_52.Run07760.0000.h5",
            "drs4_time_calibration_path" : "/fefs/aswg/data/real/DL1/20220409/v0.9/tailcut84/log/time_calibration.Run05979.0000.h5"
        },
         "PointingSource" : {
            "drive_report_path" : "/fefs/aswg/data/real/monitoring/DrivePositioning/drive_log_22_04_09.txt"
        },
         "LocalPeakWindowSum":{
            "window_shift": 4,
            "window_width": 8,
            "apply_integration_correction": False
        }
    }
    })

config_calib = Config({
  "calibration_product": "LSTCalibrationCalculator",

  "LSTCalibrationCalculator":{
    "systematic_correction_path": None,
    "squared_excess_noise_factor": 1.222,
    "flatfield_product": "FlasherFlatFieldCalculator",
    "pedestal_product": "PedestalIntegrator",
    "PedestalIntegrator":{
      "sample_size": 1,
      "sample_duration":100000,
      "tel_id":1,
      "time_sampling_correction_path": None,
      "charge_median_cut_outliers": [-10,10],
      "charge_std_cut_outliers": [-10,10],
      "charge_product":"FixedWindowSum",
      "FixedWindowSum":{
        "window_shift": 6,
        "window_width":12,
        "peak_index": 18,
        "apply_integration_correction": False
      }
    },
    "FlasherFlatFieldCalculator":{
      "sample_size": 1,
      "sample_duration":100000,
      "tel_id":1,
      "time_sampling_correction_path": None,
      "charge_product":"LocalPeakWindowSum",
      "charge_median_cut_outliers": [-0.5,0.5],
      "charge_std_cut_outliers": [-10,10],
      "time_cut_outliers": [2,38],
      "LocalPeakWindowSum":{
        "window_shift": 5,
        "window_width":12,
        "apply_integration_correction": False
      }
    }
  }})

fname_data = '/fefs/aswg/data/real/R0/20220409/LST-1.1.Run07761.0050.fits.fz'

src = EventSource(fname_data,
                  config=config_data,
                  max_events = 1000)
calibrator = CameraCalibrator(subarray=src.subarray,
                              image_extractor_type="LocalPeakWindowSum",
                              config=config_data
                              )
calib_calc = CalibrationCalculator.from_name(
            config_calib['calibration_product'],
            config=config_calib,
            subarray=src.subarray
        )
i = 0
def foo():
    for event in src:
        if (event.trigger.event_type.value == 0) or (event.trigger.event_type.value == 2):
    #         if i == 0:
    #             event.mon = deepcopy(src.r0_r1_calibrator.mon_data)
    #             i = i+1
            event.mon = deepcopy(src.r0_r1_calibrator.mon_data)
#             print('HW failing pixels')
#             print(np.argwhere(event.mon.tel[1].pixel_status.hardware_failing_pixels == True))
#             print('Ped failing pixels')
#             print(np.argwhere(event.mon.tel[1].pixel_status.pedestal_failing_pixels == True))
#             print('FF failing pixels')
#             print(np.argwhere(event.mon.tel[1].pixel_status.flatfield_failing_pixels == True))
#             print('Unusable pixels')
#             print(np.argwhere(event.mon.tel[1].calibration.unusable_pixels))
#             #print(f'Unusable pixels shape {event.mon.tel[1].calibration.unusable_pixels.shape}')
#             print('Process interleaved calibration event')
            new_ped_event, new_ff_event = calib_calc.process_interleaved(event)
            return event.mon.tel[1].calibration.unusable_pixels
#             print('Unusable pixels')
#             print(np.argwhere(event.mon.tel[1].calibration.unusable_pixels))
#             src.r0_r1_calibrator.calibrate(event)
#             calibrator(event)# calibrate till DL1
#             print('After DL1 calibration step')
#             print('Unusable pixels')
#             print(event.mon.tel[1].calibration.unusable_pixels)
            #return event.mon.tel[1].calibration.unusable_pixels
broken_pixels = foo()
        


In [ ]:
np.unique(np.where(broken_pixels==True)[1])

In [ ]:
array([  10,   87,  184,  236,  237,  284,  292,  293,  294,  295,  355,
        356,  365,  434,  438,  447,  562,  658,  659,  749,  750,  751,
        820,  827,  847,  848,  856,  902, 1008, 1010, 1148, 1154, 1183,
       1399, 1623, 1697])